In [ ]:
import os

# replace the below with your own Vectara Customer ID, Corpus ID and API Key
# In this example we point to the Corpus powering the Vectara AskFeynman demo: askfeynman.demo.vectara.com
vectara_corpus_id = os.environ['VECTARA_CORPUS_ID']
vectara_customer_id = os.environ['VECTARA_CUSTOMER_ID']
vectara_api_key = os.environ['VECTARA_API_KEY']

In [2]:
import dspy
from dspy.retrieve.vectara_rm import VectaraRM

In [3]:
# Make sure to include your OPENAI key in the environment variable OPENAI_API_KEY

llm = dspy.OpenAI(model="gpt-3.5-turbo")
retriever_model = VectaraRM()
dspy.settings.configure(lm=llm, rm=retriever_model)

In [4]:
retrieve = dspy.Retrieve(k=5)
question = "What is quantum mechanics?"
topK_passages = retrieve(question).passages
print(f"Top {retrieve.k} passages for question: {question} \n", "\n")

for idx, passage in enumerate(topK_passages):
    print(f"{idx+1}]", passage, "\n")

Top 5 passages for question: What is quantum mechanics? 
 

1] Here is one equation which
is different. In classical physics
xpx−pxx=0. What is it in quantum mechanics? ˆxˆpx−ˆpxˆx=?
Let’s work it out in the x-representation. So that we’ll know what
we are doing we put in some wave function ψ(x). 

2] (3.6), all you need to know to
continue the analysis is two numbers—in this particular case
⟨1|s⟩ and ⟨2|s⟩. These two complex numbers are
enough to predict all the future. That is what really makes quantum
mechanics easy. It turns out that in later chapters we are going to do
just such a thing when we specify a starting condition in terms of two
(or a few) numbers. Of course, these numbers depend upon where the
source is located and possibly other details about the apparatus, but
given the two numbers, we do not need to know any more about such
details. 

3] And if by having things in the way, we don’t
let it look, that we will get an analog of diffraction? The miracle of
it all is, of c

In [5]:
class RAG(dspy.Module):
    def __init__(self, num_passages=5):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [6]:
rag = RAG()
q1 = "is light a wave or particle?"
rag(q1).answer

'Light exhibits properties of both waves and particles. Initially, it was believed to behave solely as a wave, as described by Newton. However, further experiments and observations revealed that light can also behave like a particle. This duality is known as wave-particle duality. So, the answer to the question is that light is both a wave and a particle.'

In [7]:
rag = RAG()
q2 = "What is quantum mechanics?"
rag(q2).answer

"Quantum mechanics is a theory that describes the behavior of particles at the microscopic level. It is different from classical physics and introduces the concept of wave functions, which represent the probability distribution of a particle's properties. In quantum mechanics, particles can exist in multiple states simultaneously and can exhibit wave-particle duality. The theory also includes principles such as superposition, entanglement, and uncertainty."